In [4]:
import logging
import warnings
import time
import numpy as np
import gc
from concurrent.futures import ProcessPoolExecutor
from comparison_models import *
from perturb_experiment_helper import *

logging.getLogger('tensorflow').setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

# target function that runs experiments and saves data (in seperate/unique files)
"""def perturbation_experiment(trial_number: int, 
                                  input_dimension: int, 
                                  epochs: int = 20, 
                                  batch_size: int = 128, 
                                  use_pseudorehearsal: bool = False,
                                  loss_function: str = "mse",
                                  optimizer: str = "sgd",
                                  verbose: int = 0) -> None:"""
    '''Run a basic perturbation experiment with various configurations.
    
    Args:
    - trial_number (int): Identifier for the trial.
    - input_dimension (int): Dimension of the input space.
    - epochs (int, optional): Number of training epochs. Default is 20.
    - batch_size (int, optional): Size of training batch. Default is 128.
    - pseudo_rehearse (bool, optional): Whether to use pseudorehearsal. Default is False.
    - loss_function (str, optional): Loss function for training. Default is "mse".
    - verbose (int, optional): Verbosity mode for training. Default is 0 (silent).
    '''

start_time = time.time()

optimizers = ['adam', 'sgd']
pseudorehearsals = [True, False]
trial_numbers = list(range(6))
input_dimensions = [x for x in range(1,3)]

# Split the trial_numbers into 3 roughly equal parts
subsets_of_trials = np.array_split(trial_numbers, 3)

for trial_subset in subsets_of_trials:
    all_experiments = []

    #Something that contains all the combinations of parameters for the function

    # Execute all experiments in parallel
    with ProcessPoolExecutor() as executor:
        executor.map(#target function, experiment parameters)

    # Cleanup to save memory
    del all_experiments
    gc.collect()

print(f"Execution Time: {time.time() - start_time} seconds")


Execution Time: 0.6305534839630127 seconds


In [ ]:
# Standard library imports
#import os
import logging
import warnings
#import gc
#import time

# Third-party library imports
#import numpy as np
#import tensorflow as tf

# Local module imports
from comparison_models import *
from perturb_experiment_helper import *

# Configuration for logging and warnings
logging.getLogger('tensorflow').setLevel(logging.ERROR)
warnings.filterwarnings('ignore')

import time
import numpy as np
import os
import gc
from concurrent.futures import ProcessPoolExecutor


def wrapper(args):
    basic_perturbation_experiment(**args)

def load_and_save_data(input_dim=2, 
                       use_pseudorehearsal=False, 
                       optimizer='sgd', 
                       trials=30, 
                       num_models=6):
    """
    Load and save data from specified trials and models to a folder.
    
    Parameters:
    - input_dim (int): Input dimension.
    - use_pseudorehearsal (bool): Whether pseudo rehearsal is used.
    - optimizer (str): Type of optimizer.
    - trials (int): Number of trials.
    - num_models (int): Number of models.
    """
    base_folder = f"results/input_dim_{input_dim}_{use_pseudorehearsal}_{optimizer}"

    # Initialize the main data storage with the desired structure
    data = {
        "min_distance": [],
        "max_distance": []
    }

    for j in range(num_models):
        data[f"model_{j}_perturbations"] = []

    for i in range(trials):
        trial_folder = f"{base_folder}/trial_{i}"

        min_distance_path = f"{trial_folder}/distances/min_distances.npy"
        max_distance_path = f"{trial_folder}/distances/max_distances.npy"

        # Load the distances from the numpy files
        if os.path.exists(min_distance_path):
            min_distance = np.load(min_distance_path)
            data["min_distance"].append(min_distance)
        else:
            print(f"Warning: {min_distance_path} not found.")

        if os.path.exists(max_distance_path):
            max_distance = np.load(max_distance_path)
            data["max_distance"].append(max_distance)
        else:
            print(f"Warning: {max_distance_path} not found.")

        for j in range(num_models):
            perturbation_path = f"{trial_folder}/perturbations/model_{j}/absolute_perturbation.npy"

            # Load the perturbation from the numpy file
            if os.path.exists(perturbation_path):
                perturbation_data = np.load(perturbation_path)
                data[f"model_{j}_perturbations"].append(perturbation_data)
            else:
                print(f"Warning: {perturbation_path} not found.")

    # Convert lists of numpy arrays to a single concatenated numpy array
    for key in data:
        if data[key]:  # Check if the list is not empty
            data[key] = np.concatenate(data[key], axis=0).flatten()
        else:
            data[key] = None  # Set the value to None if the list is empty

    # Save the data
    save_folder = "aggregated_results"
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
    save_path = f"{save_folder}/data_input_dim_{input_dim}.npy"
    np.save(save_path, data)

def prepare_experiments(args):
    input_dim, trial_number, optimizers, pseudorehearsals = args
    experiments = []
    #for trial in trial_numbers:
    for optimizer in optimizers:
        for use_pseudorehearsal in pseudorehearsals:
            experiment_arguments = {
                'trial_number': trial_number,
                'input_dimension': input_dim,
                'use_pseudorehearsal': use_pseudorehearsal,
                'loss_function': 'mse',
                'verbose': 0,
                'optimizer': optimizer
            }
            experiments.append(experiment_arguments)
    return experiments

#if __name__ == "__main__":
start_time = time.time()

#trial_numbers = list(range(31))
optimizers = ['adam', 'sgd']
pseudorehearsals = [True, False]
number_of_trials = 3
input_dimensions = [x for x in range(1,3)]


for trial_num in range(1, number_of_trials): # Iterate over partition numbers using a for-loop
    input_dim_combinations = [(input_dim, trial_num, optimizers, pseudorehearsals) 
                              for input_dim in range(1, 7)]
    print(input_dim_combinations)

    all_experiments = []
    for combo in input_dim_combinations:
        all_experiments.extend(prepare_experiments(combo))

    # Execute all experiments for the given partition in parallel
    with ProcessPoolExecutor() as executor:
        executor.map(wrapper, all_experiments)

    # Aggregate results after all experiments for the given partition have run
    for combo in input_dim_combinations:
        input_dim, _, _, _ = combo

        # Loop through optimizers and pseudorehearsals to call the `load_and_save_data` for each combination
        for optimizer in optimizers:
            for use_pseudorehearsal in pseudorehearsals:
                load_and_save_data(input_dim=input_dim, 
                                   optimizer=optimizer, 
                                   use_pseudorehearsal=use_pseudorehearsal, 
                                   trials=number_of_trials, 
                                   num_models=18)

    # Cleanup to save memory
    del all_experiments
    gc.collect()

print(f"Execution Time: {time.time() - start_time} seconds")



In [2]:
perturbation_experiment(trial_number=2, input_dimension=2)

INFO:tensorflow:Assets written to: results/input_dim_2_False_sgd/trial_2/models/model_0\assets
INFO:tensorflow:Assets written to: results/input_dim_2_False_sgd/trial_2/models/model_1\assets
INFO:tensorflow:Assets written to: results/input_dim_2_False_sgd/trial_2/models/model_2\assets
INFO:tensorflow:Assets written to: results/input_dim_2_False_sgd/trial_2/models/model_3\assets


INFO:tensorflow:Assets written to: results/input_dim_2_False_sgd/trial_2/models/model_4\assets


INFO:tensorflow:Assets written to: results/input_dim_2_False_sgd/trial_2/models/model_4\assets


INFO:tensorflow:Assets written to: results/input_dim_2_False_sgd/trial_2/models/model_5\assets


INFO:tensorflow:Assets written to: results/input_dim_2_False_sgd/trial_2/models/model_5\assets


...

INFO:tensorflow:Assets written to: results/input_dim_2_False_sgd/trial_2/models/model_16\assets


INFO:tensorflow:Assets written to: results/input_dim_2_False_sgd/trial_2/models/model_17\assets


INFO:tensorflow:Assets written to: results/input_dim_2_False_sgd/trial_2/models/model_17\assets


INFO:tensorflow:Assets written to: results/input_dim_2_False_sgd/trial_2/models/model_18\assets


INFO:tensorflow:Assets written to: results/input_dim_2_False_sgd/trial_2/models/model_18\assets
